In [5]:
# Read the 23andMe raw data file and turn it into a dataframe

import pandas as pd
import numpy as np
import sys

ExampleData = "../../genome_Chad_Wrye_v5_Full_20220921063742.txt"
# Read the text file into a DataFrame
ttAndMe = pd.read_csv(ExampleData, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])



/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_19006/328209591.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(ExampleData, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])


In [7]:
# Read the PRS313.xlsx file into a DataFrame
PRS313 = pd.read_excel('../../PRS313.xlsx')

PRS313

,SNPa,Chromosome,Positionb,Reference Allele,Effect Allele,EAFc,Overall Breast Cancerd,ER-positivee,ER-negativef,hybrid ER-positiveg,hybrid ER-negativeh
0,1_100880328_A_T,1,100880328,A,T,0.409678,0.0373,0.0355,0.0160,0.0373,0.0373
1,1_10566215_A_G,1,10566215,A,G,0.329016,-0.0586,-0.0407,-0.1109,-0.0407,-0.1109
2,1_110198129_CAAA_C,1,110198129,CAAA,C,0.775478,0.0458,0.0545,0.0266,0.0458,0.0458
3,1_114445880_G_A,1,114445880,G,A,0.166363,0.0621,0.0642,0.0579,0.0621,0.0621
4,1_118141492_A_C,1,118141492,A,C,0.265682,0.0452,0.0417,0.0551,0.0452,0.0452
...,...,...,...,...,...,...,...,...,...,...,...
308,22_39343916_T_A,22,39343916,T,A,0.254083,0.0407,0.0326,0.0330,0.0407,0.0407
309,22_40904707_CT_C,22,40904707,CT,C,0.109871,0.1148,0.1160,0.1203,0.1148,0.1148
310,22_43433100_C_T,22,43433100,C,T,0.114445,-0.0600,-0.0585,-0.0515,-0.0600,-0.0600
311,22_45319953_G_A,22,45319953,G,A,0.416619,-0.0134,-0.0060,-0.0611,-0.0060,-0.0611


In [8]:
# Find all the positions in ttAndMe that are in PRS313 by identifying columns with the same position and chromosome

PRS313['in_23andMe'] = PRS313.apply(lambda x: ttAndMe[(ttAndMe['position'] == x['Positionb']) & (ttAndMe['chromosome'] == x['Chromosome'])].shape[0] > 0, axis=1)

In [11]:
PRS313[(PRS313['in_23andMe'] == True) & (PRS313["Chromosome"] == 1)]

,SNPa,Chromosome,Positionb,Reference Allele,Effect Allele,EAFc,Overall Breast Cancerd,ER-positivee,ER-negativef,hybrid ER-positiveg,hybrid ER-negativeh,in_23andMe
1,1_10566215_A_G,1,10566215,A,G,0.329016,-0.0586,-0.0407,-0.1109,-0.0407,-0.1109,True
5,1_120257110_T_C,1,120257110,T,C,0.530949,0.0385,0.0430,0.0226,0.0430,0.0226,True
6,1_121280613_A_G,1,121280613,A,G,0.405315,0.0881,0.1052,0.0209,0.1052,0.0209,True
9,1_149906413_T_C,1,149906413,T,C,0.401747,0.0548,0.0625,0.0388,0.0548,0.0548,True
13,1_18807339_T_C,1,18807339,T,C,0.514474,-0.0564,-0.0649,-0.0248,-0.0649,-0.0248,True
14,1_201437832_C_T,1,201437832,C,T,0.055937,0.0917,0.0815,0.0714,0.0917,0.0917,True
20,1_217220574_G_A,1,217220574,G,A,0.210653,-0.0440,-0.0459,0.0029,-0.0459,0.0029,True
22,1_242034263_A_G,1,242034263,A,G,0.030504,0.1428,0.1519,0.1413,0.1428,0.1428,True
23,1_41380440_C_T,1,41380440,C,T,0.643842,0.0426,0.0423,0.0395,0.0426,0.0426,True
28,1_88156923_G_A,1,88156923,G,A,0.148676,0.0494,0.0580,0.0183,0.0580,0.0183,True


In [12]:
PRS313.to_excel('../../Data/PRS313_with_23andMe.xlsx', index=False)